In [ ]:
### Import libraries
from netCDF4 import Dataset
import xarray as xr
import numpy as np
import cartopy.crs as crs
import matplotlib
from matplotlib.cm import get_cmap
import matplotlib.pyplot as plt
from wrf import getvar, interplevel, smooth2d, destagger
import warnings
from scipy.interpolate import griddata
from matplotlib.cm import get_cmap
import cartopy.crs as crs
from cartopy.feature import NaturalEarthFeature
warnings.filterwarnings('ignore')
import matplotlib.transforms as mtransforms
import matplotlib.ticker as mticker

CP = 1005.7
RD = 287.04
P0 = 1000.
TR = 300.
LV = 2.501e6
EPS = 1.


In [ ]:
### COAMPS settings
nx_c = 301
ny_c = 201
nz_c = 45
coamps_dir = '/p/work1/lloveras/real/nov2018/coamps_files/'

### Latitude and longitude
lats = np.fromfile(coamps_dir + 'latitu_sfc_000000_000000_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
lons = np.fromfile(coamps_dir + 'longit_sfc_000000_000000_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
lats = np.reshape(lats, [ny_c, nx_c])
lons = np.reshape(lons, [ny_c, nx_c])
lons[lons > 0] -= 360

### Full pressure in hPa
p_c = (np.fromfile(coamps_dir + '2x/ttlprs_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4') \
      + np.fromfile(coamps_dir + 'ttlprs_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4'))/2
p_c = np.flip(np.reshape(p_c, [nz_c, ny_c, nx_c]), axis=0)

### Theta
t_c = np.fromfile(coamps_dir + '2x/pottmp_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4') \
        - np.fromfile(coamps_dir + 'pottmp_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
t_c = np.flip(np.reshape(t_c, [nz_c, ny_c, nx_c]),axis=0)

### Wind
u_c = np.fromfile(coamps_dir + '2x/uuwind_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4') \
        - np.fromfile(coamps_dir + 'uuwind_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
v_c = np.fromfile(coamps_dir + '2x/vvwind_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4') \
        - np.fromfile(coamps_dir + 'vvwind_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
w_c = np.fromfile(coamps_dir + '2x/wwwind_sig_029735_000000_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4') \
        - np.fromfile(coamps_dir + 'wwwind_sig_029735_000000_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
u_c = np.flip(np.reshape(u_c, [nz_c, ny_c, nx_c]),axis=0)
v_c = np.flip(np.reshape(v_c, [nz_c, ny_c, nx_c]),axis=0)
w_c = np.flip(np.reshape(w_c, [nz_c+1, ny_c, nx_c]),axis=0)
w_c_ds = destagger(w_c,stagger_dim=0)

### Water vapor
qv_c = np.fromfile(coamps_dir + '2x/wvapor_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4') \
        - np.fromfile(coamps_dir + 'wvapor_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
qv_c = np.flip(np.reshape(qv_c, [nz_c, ny_c, nx_c]),axis=0)


### Accumulated perturbations

### Full pressure in hPa
p_a = (np.fromfile(coamps_dir + '9x/ttlprs_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4') \
      + np.fromfile(coamps_dir + 'ttlprs_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4'))/2
p_a = np.flip(np.reshape(p_a, [nz_c, ny_c, nx_c]), axis=0)

### Theta
t_a = np.fromfile(coamps_dir + '9x/pottmp_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4') \
        - np.fromfile(coamps_dir + 'pottmp_sig_028485_000010_1a0301x0201_2018111312_00000000_fcstfld', dtype='>f4')
t_a = np.flip(np.reshape(t_a, [nz_c, ny_c, nx_c]),axis=0)

fac = np.amax(np.abs(t_a))/np.amax(np.abs(t_c))
t_c = t_c*fac


In [ ]:
dt850 = np.nan_to_num(interplevel(t_a, p_a, 800))
dt700 = np.nan_to_num(interplevel(t_a, p_a, 600))
dt500 = np.nan_to_num(interplevel(t_a, p_a, 500))
dt300 = np.nan_to_num(interplevel(t_a, p_a, 300))


In [ ]:
### Contour interval
zlvls = np.asarray([-9,-7,-5,-3,-1,1,3,5,7,9])/5

### Plot subdomain
min_lat = 15
max_lat = 60
min_lon = -135
max_lon = -65

### Cartopy projection
proj = crs.LambertConformal(central_longitude=-100,standard_parallels=(20,70))

fig, axd = plt.subplot_mosaic([['topleft','topright','cbar'],['bottomleft','bottomright','cbar']],
                              constrained_layout=True, figsize=(7.4,4.8), dpi=2000, 
                              gridspec_kw={'width_ratios':[1,1,0.05],'height_ratios':[1,1]},
                              per_subplot_kw={'topleft':{'projection':proj},
                                              'topright':{'projection':proj},
                                              'bottomleft':{'projection':proj},
                                              'bottomright':{'projection':proj}})

trans = mtransforms.ScaledTranslation(1/30, -1/30, fig.dpi_scale_trans)

##############
### TOP LEFT
#############

# Add the gridlines
gls = axd['topleft'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topleft'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.bottom_labels=False
gls.rotate_labels = False

im1 = axd['topleft'].contourf(lons, lats, dt300, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topleft'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topleft'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topleft'].set_title('300 hPa')
axd['topleft'].text(0.0, 1.0, '(a)',transform=axd['topleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### TOP RIGHT
#############

gls = axd['topright'].gridlines(draw_labels=False, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['topright'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])

im1 = axd['topright'].contourf(lons, lats, dt500, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['topright'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['topright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['topright'].set_title('500 hPa')
axd['topright'].text(0.0, 1.0, '(b)',transform=axd['topright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM LEFT
#############

gls = axd['bottomleft'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottomleft'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.rotate_labels = False

im1 = axd['bottomleft'].contourf(lons, lats, dt700, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomleft'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottomleft'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['bottomleft'].set_title('600 hPa')
axd['bottomleft'].text(0.0, 1.0, '(c)',transform=axd['bottomleft'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

##############
### BOTTOM RIGHT
#############

gls = axd['bottomright'].gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False,linestyle='dashed')
axd['bottomright'].set_extent([min_lon, max_lon, min_lat, max_lat],crs=crs.PlateCarree())
gls.xlocator = mticker.FixedLocator([-180,-160, -140, -120, -100, -80, -60, -40,-20])
gls.ylocator = mticker.FixedLocator([0, 20, 40, 60, 80])
gls.top_labels=False
gls.right_labels=False
gls.left_labels=False
gls.rotate_labels = False

im1 = axd['bottomright'].contourf(lons, lats, dt850, levels=zlvls,
            cmap = get_cmap('RdBu_r'), extend='both',
            transform=crs.PlateCarree())

# Download and add the states and coastlines
states = NaturalEarthFeature(category='cultural', scale='50m',
                             facecolor='none',
                             name='admin_1_states_provinces')
axd['bottomright'].add_feature(states, linewidth=.5, edgecolor='k',alpha=0.5)
axd['bottomright'].coastlines('50m', linewidth=0.8,color='k',alpha=0.5)

# Title
axd['bottomright'].set_title('800 hPa')
axd['bottomright'].text(0.0, 1.0, '(d)',transform=axd['bottomright'].transAxes + trans,
            fontsize=12,verticalalignment='top',
            bbox=dict(facecolor='white', edgecolor='k', pad=2),zorder=20)

### SET THE COLORBAR AND SHOW
cbar = fig.colorbar(im1, orientation='vertical', cax=axd['cbar'])
plt.show()
